<a href="https://colab.research.google.com/github/manelmech/Evaluation-of-transformer-based-methods/blob/main/Fine_tuning_bert_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment setup

In [1]:
!nvidia-smi


Thu Jan  5 15:59:14 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P0    29W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install transformers

!pip install sentencepiece transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 99.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.1 MB/s eta 0:00:00


In [3]:
 pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 52.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4


In [12]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

from tabulate import tabulate
from tqdm import trange
import random

# Dataset

In [7]:
from google.colab import drive
drive.mount('/content/drive')


df = pd.read_csv('/content/drive/My Drive/GCDC_train.csv')

df_test = pd.read_csv('/content/drive/My Drive/GCDC_test.csv')

Mounted at /content/drive


In [8]:
from sklearn.utils import shuffle
df = shuffle(df)
df['labelA']= df['labelA'].astype(int)
df['labelA'] =df['labelA'] - 1
df_test['labelA']= df_test['labelA'].astype(int)
df_test['labelA'] =df_test['labelA'] - 1
print(df['labelA'])

1161    0
475     2
1093    2
2696    0
882     1
       ..
602     2
73      2
1614    2
3660    2
792     1
Name: labelA, Length: 4000, dtype: int64


In [9]:
text = df.text.values
labels = df.labelA.values

text_eval = df_test.text.values
labels_eval = df_test.labelA.values


# Preprocessing

In [13]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case = True)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [14]:
token_id = []
attention_masks = []
token_id_eval = []
attention_masks_eval = []

def preprocessing(input_text, tokenizer):
  '''
  Returns <class transformers.tokenization_utils_base.BatchEncoding> with the following fields:
    - input_ids: list of token ids
    - token_type_ids: list of token type ids
    - attention_mask: list of indices (0,1) specifying which tokens should considered by the model (return_attention_mask = True).
  '''
  return tokenizer.encode_plus(
                        input_text,
                        add_special_tokens = True,
                        max_length = 400,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt'
                   )


for sample in text:
  encoding_dict = preprocessing(sample, tokenizer)
  token_id.append(encoding_dict['input_ids']) 
  attention_masks.append(encoding_dict['attention_mask'])


token_id = torch.cat(token_id, dim = 0)
attention_masks = torch.cat(attention_masks, dim = 0)
labels = torch.tensor(labels)


for sample in text_eval:
  encoding_dict_eval = preprocessing(sample, tokenizer)
  token_id_eval.append(encoding_dict_eval['input_ids']) 
  attention_masks_eval.append(encoding_dict_eval['attention_mask'])


token_id_eval = torch.cat(token_id_eval, dim = 0)
attention_masks_eval = torch.cat(attention_masks_eval, dim = 0)
labels_eval = torch.tensor(labels_eval)



Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


# Data split

In [15]:
val_ratio = 0.2
# Recommended batch size: 16, 32. See: https://arxiv.org/pdf/1810.04805.pdf
batch_size = 8

# Indices of the train and validation splits stratified by labels


# Train and validation sets
train_set = TensorDataset(token_id, 
                          attention_masks, 
                          labels)

val_set = TensorDataset(token_id_eval, 
                        attention_masks_eval, 
                        labels_eval)

# Prepare DataLoader
train_dataloader = DataLoader(
            train_set,
            sampler = RandomSampler(train_set),
            batch_size = batch_size
        )

validation_dataloader = DataLoader(
            val_set,
            sampler = SequentialSampler(val_set),
            batch_size = batch_size
        )

# Train

In [16]:
# Load the BertForSequenceClassification model
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = 3,
    output_attentions = False,
    output_hidden_states = False,
)

# Recommended learning rates (Adam): 5e-5, 3e-5, 2e-5. See: https://arxiv.org/pdf/1810.04805.pdf
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
                                {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
                                {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay':0.0}
]

optimizer = torch.optim.AdamW(model.parameters(), 
                              lr = 1e-5,
                              eps = 1e-08
                              )


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device
model = model.to(device)

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [17]:
from sklearn import metrics
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device
predictions , true_labels = [], []

# Recommended number of epochs: 2, 3, 4. See: https://arxiv.org/pdf/1810.04805.pdf
epochs = 4

for _ in trange(epochs, desc = 'Epoch'):
    
    # ========== Training ==========
    
    # Set model to training mode
    model.train()
    
    # Tracking variables
    acc = 0
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    
    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        optimizer.zero_grad()
        # Forward pass
        train_output = model(b_input_ids, 
                             token_type_ids = None, 
                             attention_mask = b_input_mask, 
                             labels = b_labels)  
        # Backward pass
        train_output.loss.backward()
        optimizer.step()
        # Update tracking variables
        tr_loss += train_output.loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        
    
    # ========== Validation ==========
    nb_tr_eval = 0
    # Set model to evaluation mode
    model.eval()

    
    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
          # Forward pass
          eval_output = model(b_input_ids, 
                              token_type_ids = None, 
                              attention_mask = b_input_mask)
        b_labels = b_labels.cpu().detach().numpy()
        logits = eval_output.logits.detach().cpu().numpy()
       
        preds = np.argmax(logits, axis = 1).flatten()
        b_labels= b_labels.flatten()
        
        predictions.extend(preds)
       # print(predictions)
       # Store predictions and true labels
       # predictions.append(logits)
        true_labels.extend(b_labels)

        accuracy = metrics.accuracy_score(b_labels,preds )
        acc += accuracy
       
        nb_tr_eval += 1

    
    print('\n\t - Accuracy: {:.4f}'.format(acc / len(validation_dataloader)))
    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))

Epoch:  25%|██▌       | 1/4 [05:14<15:43, 314.58s/it]


	 - Accuracy: 0.5825

	 - Train loss: 0.9547


Epoch:  50%|█████     | 2/4 [10:27<10:27, 313.67s/it]


	 - Accuracy: 0.5988

	 - Train loss: 0.8248


Epoch:  75%|███████▌  | 3/4 [15:40<05:13, 313.50s/it]


	 - Accuracy: 0.5825

	 - Train loss: 0.6694


Epoch: 100%|██████████| 4/4 [20:53<00:00, 313.48s/it]


	 - Accuracy: 0.5625

	 - Train loss: 0.4564


In [18]:
print(true_labels)
print(predictions)

[2, 2, 2, 2, 1, 0, 2, 1, 2, 2, 1, 2, 2, 2, 2, 1, 1, 1, 0, 2, 2, 0, 1, 2, 0, 2, 0, 0, 2, 1, 2, 0, 0, 0, 2, 2, 2, 2, 2, 2, 1, 2, 2, 0, 0, 1, 2, 2, 1, 0, 0, 1, 2, 2, 1, 0, 2, 2, 2, 2, 2, 0, 0, 0, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 2, 0, 2, 0, 2, 1, 0, 2, 1, 0, 2, 2, 1, 0, 2, 0, 2, 2, 0, 0, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 1, 0, 2, 1, 1, 1, 2, 2, 1, 0, 1, 2, 0, 0, 2, 0, 2, 2, 2, 1, 2, 0, 2, 2, 2, 0, 1, 2, 1, 1, 2, 0, 2, 2, 1, 2, 2, 2, 0, 2, 2, 2, 0, 0, 0, 1, 0, 1, 0, 2, 0, 2, 2, 2, 2, 2, 1, 2, 0, 2, 2, 2, 2, 2, 0, 0, 0, 2, 1, 1, 1, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2, 0, 1, 1, 2, 2, 2, 1, 2, 1, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 1, 1, 0, 0, 2, 2, 1, 2, 2, 2, 0, 2, 2, 1, 1, 1, 1, 2, 2, 1, 2, 2, 1, 0, 1, 2, 2, 2, 1, 1, 0, 0, 2, 2, 0, 0, 1, 1, 1, 0, 2, 2, 0, 0, 0, 2, 0, 1, 2, 0, 2, 0, 2, 2, 0, 0, 1, 2, 0, 1, 0, 2, 2, 0, 1, 1, 2, 2, 2, 0, 0, 1, 2, 2, 2, 2, 2, 0, 2, 1, 0, 2, 0, 2, 1, 2, 1, 0, 0, 1, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 0, 1, 2, 1, 1, 1, 1, 2, 2, 

In [19]:
from sklearn.metrics import accuracy_score
from sklearn import metrics

matrix = metrics.confusion_matrix(true_labels, predictions, labels=[0, 1, 2])
print(matrix)
sum = np.sum(matrix)
acc_low = (matrix[0][0] + matrix[1][1] + matrix[1][2] + matrix[2][1] + matrix [2][2])/(sum) #exactitude relative à la classe low
acc_medium = (matrix[0][0] + matrix[1][1] + matrix[0][2] + matrix[2][0] + matrix [2][2])/(sum) #exactitude relative à la classe medium
acc_high = (matrix[0][0] + matrix[1][1] + matrix[0][1] + matrix[1][0] + matrix [2][2])/(sum) #exactitude relative à la classe high

accuracy = accuracy_score(predictions, true_labels)
print("Accuracy :")
print(accuracy)
print("Accuracy low :")
print(acc_low)  
print("Accuracy medium :")
print(acc_medium) 
print("Accuracy high :")
print(acc_high) 
print("Average accuracy :")
print((acc_low + acc_medium + acc_high)/3) 
print("Classification report :")
print(metrics.classification_report(true_labels, predictions, labels=[0, 1, 2], zero_division=1))

[[ 462   71  447]
 [ 137   58  489]
 [ 120   75 1341]]
Accuracy :
0.5815625
Accuracy low :
0.7578125
Accuracy medium :
0.75875
Accuracy high :
0.6465625
Average accuracy :
0.7210416666666667
Classification report :
              precision    recall  f1-score   support

           0       0.64      0.47      0.54       980
           1       0.28      0.08      0.13       684
           2       0.59      0.87      0.70      1536

    accuracy                           0.58      3200
   macro avg       0.51      0.48      0.46      3200
weighted avg       0.54      0.58      0.53      3200

